In [19]:
#Using the same code i used from HW3 for the data Preprocessing

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import svm


# get Training and Testing sets
train_set = pd.read_csv('./input/train.csv')
test_set = pd.read_csv('./input/test.csv')

combine = [train_set, test_set]

# Store our passenger ID for easy access
PassengerId = test_set['PassengerId']

In [20]:
#drop columns not needed
train_set = train_set.drop(["Ticket", "Cabin"], axis=1)
test_set = test_set.drop(["Ticket", "Cabin"], axis=1)
combine = [train_set, test_set]

# change sex to binary
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    
train_set.fillna(train_set.median(), inplace=True)
test_set.fillna(test_set.median(), inplace=True)

#Fill empty/null values for embarked
freq_port = train_set.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
#fill empty/null values for fare
test_set['Fare'].fillna(test_set['Fare'].dropna().median(), inplace=True)    

#update combine
combine = [train_set, test_set]

In [21]:
#change fare to integers so it can be used in model easier
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [22]:
#change embarked to numbers so it can be used in the model
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
#dropping name and passengerid
train_set = train_set.drop(['Name','PassengerId'], axis=1)
test_set = test_set.drop(['Name','PassengerId'], axis=1)

train_set.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,0,0
1,1,1,1,38.0,1,0,3,1
2,1,3,1,26.0,0,0,1,0
3,1,1,1,35.0,1,0,3,0
4,0,3,0,35.0,0,0,1,0


In [23]:
#dropping unnecessary columns
trainX = train_set.drop(["Survived", "SibSp", "Parch", "Embarked"], axis=1).values
trainY = train_set["Survived"].values

trainSplitX, testSplitX, trainSplitY, testSplitY = train_test_split(trainX, trainY, test_size = 0.33, random_state = 45)
trainSplitX.shape, trainSplitY.shape, testSplitX.shape, testSplitY.shape

((596, 4), (596,), (295, 4), (295,))

In [24]:
# Obtain score of Linear Kernel
linearKernel = svm.SVC(kernel="linear")
accuracyScore = cross_validate(linearKernel, trainSplitX, trainSplitY, scoring=["accuracy"])
accuracyScore["test_accuracy"].mean()

0.7701400560224089

In [25]:
# Obtain score of quadratic/polynomial Kernel
quadraticKernel = svm.SVC(kernel="poly")
accuracyScore = cross_validate(quadraticKernel, trainSplitX, trainSplitY, scoring=["accuracy"])
accuracyScore["test_accuracy"].mean()

0.6644257703081232

In [26]:
# Obtain score of RBF Kernel
rbfKernel = svm.SVC(kernel="rbf")
accuracyScore = cross_validate(rbfKernel, trainSplitX, trainSplitY, scoring=["accuracy"])
accuracyScore["test_accuracy"].mean()

0.6274789915966387